## <font color="red">Caution!</font>

Before running a Marlowe contract on `mainnet`, it is wise to do the following in order to avoid losing funds:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

---

# First Marlowe Peer-to-Peer Loan on Cardano Mainnet

This peer-to-peer uncollateralized loan consists of two payments spaced ninety days apart and uses a private stablecoin USDB.
1. On 14 November 2022, the loan contract is created: [3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21](https://cardanoscan.io/transaction/3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21?tab=utxo)
2. Before 14 November 2022, the counterparty (i.e., the lender) deposits 9000 USDB into the Marlowe contract, and the contract pays that to the party (i.e., the borrower): [5137a4eecbe9b53e1afa9cd27f10c1b8a1ad30d8459e88ca5ed198b588025da6](https://cardanoscan.io/transaction/5137a4eecbe9b53e1afa9cd27f10c1b8a1ad30d8459e88ca5ed198b588025da6?tab=utxo)
3. Before 11 February 2023, the party deposits the first installment of 4560 USDB (approximately half of the principal plus interest), which is paid to the counterparty: [19a20efac7625805e6bdcc7f3a1e5ecfb9057eaa7583cc8b8435d28906461f78](https://cardanoscan.io/transaction/19a20efac7625805e6bdcc7f3a1e5ecfb9057eaa7583cc8b8435d28906461f78?tab=utxo)
4. Before 13 May 2023, the party must deposit the last installment of 4560 USDB (the remainder of principle and interest), which is paid to the counterparty and closes the contract, completing the loan.

![Loan contract](contract.png)

## Session 1. November 14, 2022

### Record version numbers

In [1]:
marlowe-cli --version

marlowe-cli 0.0.10.0 «mainnet»


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

33a7615477ab8886792305e927e87ec09042961d


### Select mainnet

In [4]:
export CARDANO_NODE_SOCKET_PATH=cardano-node.socket

In [5]:
TREASURY=treasury

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.mainnet.address)
echo $FAUCET_ADDR

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


In [7]:
BRIAN_SKEY=$TREASURY/brian.skey
BRIAN_ADDR=addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7

In [8]:
EVAN_SKEY=
EVAN_ADDR=addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2

### View the contract

In [9]:
json2yaml evan-loan.contract

timeout: 1668495540000
timeout_continuation: close
when:
- case:
    deposits: 9000
    into_account:
      address: addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: USDB
    party:
      address: addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7
  then:
    from_account:
      address: addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7
    pay: 9000
    then:
      timeout: 1676185140000
      timeout_continuation: close
      when:
      - case:
          deposits: 4560
          into_account:
            address: addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2
          of_token:
            currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c

### Check the deadlines in the contract

In [10]:
date -d @1668412800

Mon Nov 14 01:00:00 AM MST 2022


In [11]:
date -d @1676185140

Sat Feb 11 11:59:00 PM MST 2023


In [12]:
date -d @1683961140

Sat May 13 12:59:00 AM MDT 2023


### Create the initial state of the contract

This just contains the minimum UTxO for the script UTxO.

In [13]:
json2yaml evan-loan.state

accounts:
- - - address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1


### Transaction 1. Create the contract

Bundle the state and contract information into a single `.marlowe` file.

In [14]:
marlowe-cli run initialize \
  --mainnet \
  --at-address addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l \
  --contract-file evan-loan.contract \
  --state-file    evan-loan.state \
  --out-file      evan-loan-1.marlowe \
  --print-stats


Searching for reference script at address: addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Analyze the safety of the contract.

In [15]:
marlowe-cli run analyze \
  --mainnet \
  --marlowe-file evan-loan-1.marlowe

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 4 execution paths.
- Preconditions:
    Duplicate accounts: []
    Duplicate bound values: []
    Duplicate choices: []
    Invalid account parties: []
    Invalid account tokens: []
    Invalid choice parties: []
    Invalid roles currency: false
    Non-positive account balances: []
- Role names:
    Blank role names: false
    Invalid role names: []
- Tokens:
    Invalid tokens: []
- Maximum value:
    Actual: 96
    Invalid: false
    Maximum: 5000
    Percentage: 1.92
    Unit: byte
- Minimum UTxO:
    Requirement:
      lovelace: 1288690
- Execution cost:
    Memory:
      Actual: 6605682
      Invalid: false
      Maximum: 14000000
      Percentage: 47.18344285714286
    Steps:
      Actual: 1830393886
      Invalid: false
      Maximum: 10000000000
      Percentage: 18.30393886
- Transaction size:
    Actual: 3354
 

Execute the creation transaction.

In [16]:
marlowe-cli run auto-execute \
  --mainnet \
  --marlowe-out-file evan-loan-1.marlowe \
  --change-address $FAUCET_ADDR \
  --required-signer $FAUCET_SKEY \
  --out-file evan-loan-1.txBody \
  --submit 600 \
  --print-stats


Fee: Lovelace 248881
Size: 1779 / 16384 = 10%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21


### Transaction 2. Counterparty deposits the 9000 USDB that is then lent to the party

Prepare the transaction.

In [17]:
RIGHT_NOW=$(($(date -u +%s) * 1000))
marlowe-cli run prepare \
  --deposit-account $BRIAN_ADDR \
  --deposit-party $BRIAN_ADDR \
  --deposit-amount 9000 \
  --deposit-token 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.USDB \
  --invalid-before    $((RIGHT_NOW -  1 * 60 * 1000)) \
  --invalid-hereafter $((RIGHT_NOW + 10 * 60 * 1000)) \
  --marlowe-file evan-loan-1.marlowe \
  --out-file     evan-loan-2.marlowe \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1668447916000},POSIXTime {getPOSIXTime = 1668448576999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1668447916000},POSIXTime {getPOSIXTime = 1668448576999}), txInputs = [NormalInput (IDeposit "\"addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7\"" "\"addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7\"" (Token "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d" "USDB") 9000)]}

Datum size: 1041
Payment 1
  Acccount: "\"addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7\""
  Payee: Party "\"addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2\""
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."USDB": 9000


Execute the transaction.

In [18]:
marlowe-cli run auto-execute \
  --mainnet \
  --tx-in-marlowe "3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21#1" \
  --marlowe-in-file  evan-loan-1.marlowe \
  --marlowe-out-file evan-loan-2.marlowe \
  --change-address $BRIAN_ADDR \
  --required-signer $BRIAN_SKEY \
  --out-file evan-loan-2.txBody \
  --submit 600 \
  --print-stats


Fee: Lovelace 901257
Size: 3316 / 16384 = 20%
Execution units:
  Memory: 7468762 / 14000000 = 53%
  Steps: 2053198482 / 10000000000 = 20%
5137a4eecbe9b53e1afa9cd27f10c1b8a1ad30d8459e88ca5ed198b588025da6


Examine the UTxO at the script address.

In [19]:
cardano-cli query utxo --mainnet --address $(marlowe-cli contract address --mainnet)

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5137a4eecbe9b53e1afa9cd27f10c1b8a1ad30d8459e88ca5ed198b588025da6     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3c2165748b2363dd5c651984d96e7c05edc3c7fee57fdeb8a9d8743fa3203a98"


See that the 9000 USDB has been received by the borrower.

In [37]:
cardano-cli query utxo --mainnet --address $EVAN_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5137a4eecbe9b53e1afa9cd27f10c1b8a1ad30d8459e88ca5ed198b588025da6     2        1155080 lovelace + 9000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.55534442 + TxOutDatumNone


## Session 2. February 4, 2023

Use Marlowe Runtime to continue execution of the contract.

In [1]:
CONTRACT_ID="3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21#1"

View the current state of the contract.

In [2]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21 (creation)
ContractId:      3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21#1
SlotNo:          76880686
BlockNo:         8013745
BlockId:         1894b0c19bdfdc4a1ab934f109c74c03fcfbfa29890e71e22d9d5e224330d0da
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7") (Address "addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7")
            (Token "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d" "USDB")
            (Constant 9000))
         (Pay (Address "addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7")
            (Party (Address "addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qz

### Set parameters for the transaction

In [3]:
POLICY_ID=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
TOKEN_NAME=USDB

In [4]:
BRIAN_SKEY=brian.skey
BRIAN_ADDR="$(cat brian.mainnet.address)"
echo "BRIAN_ADDR = $BRIAN_ADDR"

BRIAN_ADDR = addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7


In [5]:
EVAN_SKEY=evan.skey
EVAN_ADDR="$(cat evan.mainnet.address)"
echo "EVAN_ADDR = $EVAN_ADDR"

EVAN_ADDR = addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2


### Transaction 3. Party deposits 4560 USDB that is repaid to the counterparty

Build the transaction.

In [6]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$EVAN_ADDR" \
  --to-party "$EVAN_ADDR" \
  --currency "$POLICY_ID" \
  --token-name USDB \
  --quantity 4560 \
  --validity-lower-bound "$((1000 * ($(date -u +%s) - 2 * 60)))" \
  --validity-upper-bound "$((1000 * ($(date -u +%s) + 6 * 60)))" \
  --change-address "$EVAN_ADDR" \
  --manual-sign evan-loan-3.txBody \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

TX_3 = 19a20efac7625805e6bdcc7f3a1e5ecfb9057eaa7583cc8b8435d28906461f78


Submit the transaction.

In [7]:
marlowe-cli transaction submit \
  --mainnet \
  --tx-body-file evan-loan-3.txBody \
  --required-signer "$EVAN_SKEY" \
  --timeout 600

TxId "19a20efac7625805e6bdcc7f3a1e5ecfb9057eaa7583cc8b8435d28906461f78"


View the new state of the contract.

In [8]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21 (creation)
ContractId:      3e5d0ac37fc61e2c635cac9eccd98d5caef8b09eed3e6c6256453eaead093b21#1
SlotNo:          76880686
BlockNo:         8013745
BlockId:         1894b0c19bdfdc4a1ab934f109c74c03fcfbfa29890e71e22d9d5e224330d0da
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7") (Address "addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7")
            (Token "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d" "USDB")
            (Constant 9000))
         (Pay (Address "addr1qy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupcvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sut9jg7")
            (Party (Address "addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qz

View the payment recieved by the counterparty.

In [9]:
cardano-cli query utxo --mainnet --address $BRIAN_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
19a20efac7625805e6bdcc7f3a1e5ecfb9057eaa7583cc8b8435d28906461f78     2        1155080 lovelace + 4560 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.55534442 + TxOutDatumNone


## Session 3. Sometime before 13 May 2023